<center>
<img src="../../img/ml_theme.png">
# Дополнительное профессиональное <br> образование НИУ ВШЭ
#### Программа "Практический анализ данных и машинное обучение"
<img src="../../img/faculty_logo.jpg" height="240" width="240">
## Автор материала: старший преподаватель Факультета Компьютерных Наук ВШЭ, программист-исследователь Mail.ru Group  Юрий Кашницкий
</center>
Материал распространяется на условиях лицензии <a href="https://opensource.org/licenses/MS-RL">Ms-RL</a>. Можно использовать в любых целях, кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Занятие 9. Разреженные данные, онлайн-обучение</center>
## <center>Практика. Стохастический классификатор и идентификация пользователей по посещению веб-сайтов</center>

In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, f1_score

**Считаем данные [соревнования](https://inclass.kaggle.com/c/identify-me-if-you-can-yandex-mipt/data) в DataFrame train_df и test_df (обучающая и тестовая выборки).**

In [2]:
train_df = pd.read_csv('../../data/websites_train_sessions.csv', index_col='session_id')
test_df = pd.read_csv('../../data/websites_test_sessions.csv', index_col='session_id')

In [3]:
train_df.head()

site1                time1   site2                time2   site3  \
session_id                                                                    
1               8  2014-01-04 08:44:50    11.0  2014-01-04 08:44:50    82.0   
2             111  2014-03-18 10:33:20    78.0  2014-03-18 10:33:31   151.0   
3              11  2014-12-02 13:13:41  3187.0  2014-12-02 13:13:41   132.0   
4             668  2014-02-14 15:16:45  1965.0  2014-02-14 15:17:13   598.0   
5            1943  2014-03-17 15:19:40  1943.0  2014-03-17 15:20:10  1943.0   

                          time3   site4                time4   site5  \
session_id                                                             
1           2014-01-04 08:45:19    68.0  2014-01-04 08:45:25     8.0   
2           2014-03-18 10:33:31   111.0  2014-03-18 10:33:31  1401.0   
3           2014-12-02 13:13:42   496.0  2014-12-02 13:13:42  1969.0   
4           2014-02-14 15:20:47  1965.0  2014-02-14 15:21:13   284.0   
5           2014-03-17 15:21:40  1943.0  2014-03-17 15:22:10  1943.0   

                          time5   ...                  time6   site7  \
session_id                        ...                                  
1           2014-01-04 08:45:25   ...    2014-01-04 08:45:51  8403.0   
2           2014-03-18 10:33:31   ...    2014-03-18 10:33:32  1375.0   
3           2014-12-02 13:13:45   ...    2014-12-02 13:13:45  3187.0   
4           2014-02-14 15:21:14   ...    2014-02-14 15:21:14    38.0   
5           2014-03-17 15:22:39   ...    2014-03-17 15:22:39  1952.0   

                          time7   site8                time8   site9  \
session_id                                                             
1           2014-01-04 08:45:51   932.0  2014-01-04 08:45:53  3260.0   
2           2014-03-18 10:33:32    38.0  2014-03-18 10:33:32  1401.0   
3           2014-12-02 13:13:45    82.0  2014-12-02 13:13:46  3191.0   
4           2014-02-14 15:21:14  4451.0  2014-02-14 15:21:14  4537.0   
5           2014-03-17 15:22:41  1943.0  2014-03-17 15:22:41  1943.0   

                          time9  site10               time10 user_id  
session_id                                                            
1           2014-01-04 08:45:53     8.0  2014-01-04 08:45:53    1845  
2           2014-03-18 10:33:32    97.0  2014-03-18 10:33:34    3322  
3           2014-12-02 13:13:46  3184.0  2014-12-02 13:13:47    2003  
4           2014-02-14 15:21:15    11.0  2014-02-14 15:21:15    1373  
5           2014-03-17 15:22:42  1943.0  2014-03-17 15:22:43    1737  

[5 rows x 21 columns]

**Объединим обучающую и тестовую выборки – это понадобится, чтоб вместе потом привести их к разреженному формату.**

In [4]:
train_test_df = pd.concat([train_df, test_df])

In [34]:
len(train_df)

95319

В обучающей выборке видим следующие признаки:
    - site1 – индекс первого посещенного сайта в сессии
    - time1 – время посещения первого сайта в сессии
    - ...
    - site10 – индекс 10-го посещенного сайта в сессии
    - time10 – время посещения 10-го сайта в сессии
    - user_id – ID пользователя
    
Сессии пользователей выделены таким образом, что они не могут быть длинее получаса или 10 сайтов. То есть сессия считается оконченной либо когда пользователь посетил 10 сайтов подряд, либо когда сессия заняла по времени более 30 минут. 

**Посмотрим на статистику признаков.**

Пропуски возникают там, где сессии короткие (менее 10 сайтов). Скажем, если человек 1 января 2015 года посетил *vk.com* в 20:01, потом *yandex.ru* в 20:29, затем *google.com* в 20:33, то первая его сессия будет состоять только из двух сайтов (site1 – ID сайта *vk.com*, time1 – 2015-01-01 20:01:00, site2 – ID сайта  *yandex.ru*, time2 – 2015-01-01 20:29:00, остальные признаки – NaN), а начиная с *google.com* пойдет новая сессия, потому что уже прошло более 30 минут с момента посещения *vk.com*.

In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95319 entries, 1 to 95319
Data columns (total 21 columns):
site1      95319 non-null int64
time1      95319 non-null object
site2      93722 non-null float64
time2      93722 non-null object
site3      92339 non-null float64
time3      92339 non-null object
site4      91085 non-null float64
time4      91085 non-null object
site5      89868 non-null float64
time5      89868 non-null object
site6      88776 non-null float64
time6      88776 non-null object
site7      87755 non-null float64
time7      87755 non-null object
site8      86738 non-null float64
time8      86738 non-null object
site9      85754 non-null float64
time9      85754 non-null object
site10     84810 non-null float64
time10     84810 non-null object
user_id    95319 non-null int64
dtypes: float64(9), int64(2), object(10)
memory usage: 16.0+ MB


In [6]:
test_df.head()

site1                time1   site2                time2   site3  \
session_id                                                                    
1            1917  2014-12-04 20:36:21     NaN                  NaN     NaN   
2           20225  2014-03-04 14:08:55    27.0  2014-03-04 14:08:59     9.0   
3              71  2014-03-31 09:20:38    63.0  2014-03-31 09:20:42    64.0   
4            7338  2014-02-14 13:39:36  7338.0  2014-02-14 14:07:53  7338.0   
5           19622  2014-03-31 09:46:04    32.0  2014-03-31 09:46:12   340.0   

                          time3    site4                time4    site5  \
session_id                                                               
1                           NaN      NaN                  NaN      NaN   
2           2014-03-04 14:08:59     32.0  2014-03-04 14:09:50  20225.0   
3           2014-03-31 09:20:43     63.0  2014-03-31 09:20:43     71.0   
4           2014-02-14 14:08:36      NaN                  NaN      NaN   
5           2014-03-31 09:46:13  19634.0  2014-03-31 09:46:14   1721.0   

                          time5    site6                time6    site7  \
session_id                                                               
1                           NaN      NaN                  NaN      NaN   
2           2014-03-04 14:10:43  20258.0  2014-03-04 14:10:44  20225.0   
3           2014-03-31 09:20:43     22.0  2014-03-31 09:20:45     64.0   
4                           NaN      NaN                  NaN      NaN   
5           2014-03-31 09:46:14    327.0  2014-03-31 09:46:15     38.0   

                          time7    site8                time8    site9  \
session_id                                                               
1                           NaN      NaN                  NaN      NaN   
2           2014-03-04 14:10:45  20225.0  2014-03-04 14:11:07  20276.0   
3           2014-03-31 09:20:46     71.0  2014-03-31 09:20:47     71.0   
4                           NaN      NaN                  NaN      NaN   
5           2014-03-31 09:46:19   1721.0  2014-03-31 09:46:19    340.0   

                          time9   site10               time10  
session_id                                                     
1                           NaN      NaN                  NaN  
2           2014-03-04 14:11:08  20225.0  2014-03-04 14:11:09  
3           2014-03-31 09:20:48     70.0  2014-03-31 09:20:51  
4                           NaN      NaN                  NaN  
5           2014-03-31 09:46:20  19634.0  2014-03-31 09:46:20

In [7]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41177 entries, 1 to 41177
Data columns (total 20 columns):
site1     41177 non-null int64
time1     41177 non-null object
site2     39529 non-null float64
time2     39529 non-null object
site3     38158 non-null float64
time3     38158 non-null object
site4     37030 non-null float64
time4     37030 non-null object
site5     36049 non-null float64
time5     36049 non-null object
site6     35083 non-null float64
time6     35083 non-null object
site7     34284 non-null float64
time7     34284 non-null object
site8     33434 non-null float64
time8     33434 non-null object
site9     32633 non-null float64
time9     32633 non-null object
site10    31907 non-null float64
time10    31907 non-null object
dtypes: float64(9), int64(1), object(10)
memory usage: 6.6+ MB


**В обучающей выборке – 550 пользователей.**

In [8]:
train_df['user_id'].nunique()

550

In [9]:
train_df['user_id'].value_counts().head()

3165    3351
537     1927
3324    1534
1826    1528
1845    1452
Name: user_id, dtype: int64

**Пока для прогноза ID пользователя будем использовать только индексы посещенных сайтов. Индексы нумеровались с 1, так что заменим пропуски на нули.**

In [10]:
train_test_df_sites = train_test_df[['site1', 'site2', 'site3', 
                                     'site4','site5', 
                                     'site6','site7', 'site8', 
                                     'site9', 'site10']].fillna(0).astype('int')

In [11]:
train_test_df_sites.head(10)

site1  site2  site3  site4  site5  site6  site7  site8  site9  \
session_id                                                                  
1               8     11     82     68      8   8393   8403    932   3260   
2             111     78    151    111   1401    151   1375     38   1401   
3              11   3187    132    496   1969    504   3187     82   3191   
4             668   1965    598   1965    284    668     38   4451   4537   
5            1943   1943   1943   1943   1943   1952   1952   1943   1943   
6            2941   2951   2958   2993    162   2967     85      7   2967   
7            9996   9996    307   9996    280   9996   9996    307   9996   
8           19823  19823   1510     32  19808      8    567    567    654   
9              72      0      0      0      0      0      0      0      0   
10             32     32    561     69     32    329    329     69     69   

            site10  
session_id          
1                8  
2               97  
3             3184  
4               11  
5             1943  
6             2967  
7             9996  
8              567  
9                0  
10             329

In [32]:
train_test_df_sites.index.nunique(), len(train_test_df_sites)

(95319, 136496)

**Создайте разреженные матрицы *X_train_sparse* и *X_test_sparse* аналогично тому, как мы это делали ранее. Используйте объединенную матрицу train_test_df_sites – потом разделите обратно на обучающую и тестовую части.**

Обратите внимание на то, что в  сессиях меньше 10 сайтов  у нас остались нули, так что первый признак (сколько раз попался 0) по смыслу отличен от остальных (сколько раз попался сайт с индексом $i$). Поэтому первый столбец разреженной матрицы надо будет удалить.

**Выделите в отдельный вектор *y* ответы на обучающей выборке.**

In [71]:
y = train_df['user_id']
#X_train_test_sparse  = csr_matrix(train_test_df_sites)
train_df_sites = train_test_df_sites.ix[:95319 ,]
test_df_sites = train_test_df_sites.iloc[95319: ,]

X_train_sparse = csr_matrix(train_df_sites)
X_test_sparse = csr_matrix(test_df_sites)

**Сохраним в pickle-файлы объекты *X_train_sparse*, *X_test_sparse* и *y* (последний – в файл *kaggle_data/train_target.pkl*).**

In [72]:
import pickle
with open('../../data/kaggle_data/X_train_sparse.pkl', 'wb') as X_train_sparse_pkl:
    pickle.dump(X_train_sparse, X_train_sparse_pkl)
with open('../../data/kaggle_data/X_test_sparse.pkl', 'wb') as X_test_sparse_pkl:
    pickle.dump(X_test_sparse, X_test_sparse_pkl)
with open('../../data/kaggle_data/train_target.pkl', 'wb') as train_target_pkl:
    pickle.dump(y, train_target_pkl)

**Разобьем обучающую выборку на 2 части в пропорции 7/3.**

In [73]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_sparse, y, test_size=0.3, 
                                                     random_state=7, stratify=y.values)

**Создайте объекты sklearn.linear_model.SGDClassifier с логистической функцией потерь и с hinge loss (логистическая регрессия и линейный SVM соответственно) и параметром random_state=7. Остальные параметры оставьте по умолчанию, разве что n_jobs=-1 никогда не помешает. Обучите  модели на выборке (X_train, y_train).**

In [ ]:
sgd_logit = SGDClassifier(random_state=7,n_jobs=-1)

In [75]:
SGDClassifier?

In [ ]:
%%time
sgd_logit.fit ''' ВАШ КОД ЗДЕСЬ '''

In [ ]:
sgd_svm = SGDClassifier ''' ВАШ КОД ЗДЕСЬ '''

In [ ]:
%%time
sgd_svm.fit ''' ВАШ КОД ЗДЕСЬ '''

**Сделаем прогнозы с помощью обеих моделей на отложенной выборке (X_valid, y_valid).**

In [ ]:
logit_valid_pred = ''' ВАШ КОД ЗДЕСЬ '''
svm_valid_pred = ''' ВАШ КОД ЗДЕСЬ '''

In [ ]:
print(accuracy_score(y_valid, logit_valid_pred))
print(accuracy_score(y_valid, svm_valid_pred))

**Сделайте прогноз для тестовой выборки с помощью sgd_logit.**

In [ ]:
logit_test_pred = ''' ВАШ КОД ЗДЕСЬ '''

**Запишите ответы в файл и сделайте посылку на Kaggle. Результат, который мы только что получили, соответствует бейзлайну "SGDCLassifer" на лидерборде, **

In [ ]:
def write_to_submission_file(predicted_labels, out_file,
                             target='user_id', index_label="session_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [ ]:
write_to_submission_file ''' ВАШ КОД ЗДЕСЬ '''